<a href="https://colab.research.google.com/github/Silmar123/News_project/blob/main/TwitterAPIScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Import of the necessary libraries
import tweepy
import json
import pandas as pd
import datetime

# http://docs.tweepy.org/en/latest/api.html

We read the api keys from our config json file 

This is done so we do not expose our keys in github

In [8]:
# Values assigning for the authentication methods     
consumer_key = "oeRyEcKV5Ebfu7vPZVvtTs45H"
consumer_secret = "LNtBLVDspKep0WJ6XI7fceoT81KpSr3u51BefUUu44tOZ6rawT"
access_token = "1299838566827274242-vPAr4kIzeFfKXtnPc8TjpbFWAi8TVS"
access_token_secret = "NH4DvtNlS41YCaq0gsOdSOmqw6PWrvMv76oizYWjuxHCj"

Run the authentication methods

In [9]:
def get_api_connection(consumer_key, consumer_secret, access_token, access_token_secret):
  # Setting access tokens
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)

  # Create the API object from tweepy
  api = tweepy.API(auth, wait_on_rate_limit=True)
  return api

Define functions to handle tweepy objects easily

In [10]:
# Given a query, it returns a number of tweet objects
# We give a default of 100 but any number could be used
def get_tweets_from_query(api, query, number_of_tweets_to_get = 100):
    return tweepy.Cursor(api.search, q=query).items(number_of_tweets_to_get)  

# Given a tweet obj we return its number of retweets
def get_rts_from_tweet(api, tweet):
    return api.get_status(tweet.id).retweet_count

# Given a tweet obj we return its username
def get_user_from_tweet(api, tweet):
    return api.get_status(tweet.id).user.screen_name

# Given a tweet obj we return its likes/favs number
def get_likes_from_tweet(api, tweet):
    return api.get_status(tweet.id).favorite_count

In [11]:
# Class for better analysis with only the needed parameters
class Tweet_for_analysis:
    text: ""
    rts: 0
    id: 0
    user: ""
    likes: 0
    time: datetime.datetime.now

    # Constructor
    def __init__(self, api, tweetObj):
        self.text = tweetObj.text
        self.rts =  get_rts_from_tweet(api, tweetObj)
        self.id = tweetObj.id
        self.user = get_user_from_tweet(api, tweetObj)
        self.likes = get_likes_from_tweet(api, tweetObj)
        self.time = tweetObj.created_at



In [12]:
# # We create an empty list 
# tweet_list = []

# # We get the tweet objects from the query 
# tom_brady_tweets = get_tweets_from_query("Tom Brady")

# # We iterate over the differetn 
# for tweet in tom_brady_tweets:
#     new_tweet = Tweet_for_analysis(tweet)
#     tweet_list.append(new_tweet)


In [13]:
# df = pd.DataFrame(
#     ([tweet.id, tweet.text, tweet.rts, tweet.user, tweet.likes, tweet.time] for tweet in tweet_list), 
#     columns = ["tweet_id","text","rt_count", "username", "likes_count", "created_at"]
# )
# df.tail()

In [14]:
def get_tweet_df_from_query(twitter_consumer_key, 
                            twitter_consumer_secret, 
                            twitter_access_token, 
                            twitter_access_token_secret, 
                            query, 
                            n_tweets = 50
                            ):
  api = get_api_connection(consumer_key,consumer_secret,access_token,access_token_secret)
  new_tweet_list = []
  tweets_received = get_tweets_from_query(api, query)
  for tweet in tweets_received:
    new_tweet_to_add = Tweet_for_analysis(api, tweet)
    new_tweet_list.append(new_tweet_to_add)
  
  df = pd.DataFrame(
    ([tweet.id, tweet.text, tweet.rts, tweet.user, tweet.likes, tweet.time] for tweet in new_tweet_list), 
    columns = ["tweet_id","text","rt_count", "username", "likes_count", "created_at"]
  )

  return df

In [15]:
df_pd = get_tweet_df_from_query(consumer_key,consumer_secret,access_token,access_token_secret,"david bisbal")

In [16]:
df_pd.tail()

,tweet_id,text,rt_count,username,likes_count,created_at
95,1322854976398438401,Now playing India Martinez - Olvide Respirar f...,0,DESIFOTOLISBOA,0,2020-11-01 10:56:18
96,1322854826397650944,David Bisbal^TITLE=Bésame^ALBUM=Bésame^YEAR,0,limitefm,0,2020-11-01 10:55:42
97,1322851001238540288,-Hoy en día ser rebelde es ir en el coche de p...,0,JerezNono,2,2020-11-01 10:40:30
98,1322848348433469440,Más a gusto que en brazos de David Bisbal: así...,0,cuore,4,2020-11-01 10:29:58
99,1322834812076515328,Perdón David Bisbal - Greeicy David Bisbal - G...,0,playzradio,0,2020-11-01 09:36:10


Save data

In [17]:
#Create an appropiate folder

hora_madrid_delta = 2

anadir_hora = datetime.timedelta(hours = hora_madrid_delta)

now = datetime.datetime.now()
now = now+anadir_hora#local time
now= now.strftime("%d_%m_%Y__%Hh_%Mm_%Ss")
now_hour=now[:15]


import os

path = "/content/drive/My Drive/Colab Notebooks/UOC_B/twitter_tweets_"+str(now_hour)

if not os.path.exists(path):
    os.makedirs(path)

#Write dataframe to csv
df_pd.to_csv(path+"/twitter_tweets_"+str(now)+".csv", index = False)